In [35]:
# Advent of Code 2023
# Day 19 Problem 1
import re
from collections import Counter
import functools

with open("aoc_19_input.txt") as f:
   A =  f.read().strip()

# c+p test case here
TEST = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}""".strip()

In [36]:
from collections import defaultdict

# NOTE: come back and do this in a graph theoretic way in the future
def parse(inp):
    flows,raw_parts = [half.split("\n") for half in inp.strip().split("\n\n")]
    workflows = defaultdict()
    for flow in flows:
        # px{a<2006:qkq,m>2090:A,rfg}
        label, rules = flow[:-1].split("{")
        tmp = []
        for rule in rules.split(","):
            if ":" in rule:
                regex = re.search(r"(?P<categ>.*)(?P<comp>[><])(?P<quant>[0-9]+):(?P<nxt>.*)", rule)
                tmp.append([regex.group("categ"), regex.group("comp"), int(regex.group("quant")), regex.group("nxt")])
            else:
                tmp.append([rule])
        workflows[label] = tmp
    
    parts = defaultdict(lambda: defaultdict())
    for i, part in enumerate(raw_parts):
        for categ in part[1:-1].split(","):
            cat, quant = categ.split("=")
            parts[i][cat] = int(quant)
    
    return workflows, parts

def checkPart(workflows, part):
    currLabel = "in"
    while True:
        if currLabel == 'A':
            return 1
        if currLabel == 'R':
            return 0
        for flow in workflows[currLabel]:
            if flow == "A": 
                return 1
            if flow == "R":
                return 0
            if len(flow) == 1:
                currLabel = flow[0]
                break
            categ, comp, quant, nxt = flow
            #print(flow, part)
            if (comp == ">" and part[categ] > quant) or (comp == "<" and part[categ] < quant):
                currLabel = nxt
                break
            currLabel = nxt

def p1(inp):
    W, P = parse(inp)
    res = 0
    for idx in P:
        #print(P[idx])
        if checkPart(W, P[idx]):
            res += P[idx]['x'] + P[idx]['m'] + P[idx]['a'] + P[idx]['s']
    return res


In [37]:
# expected output: 19114
p1(TEST)

19114

In [38]:
p1(A)

353046

In [49]:
import copy

def parseP2(inp):
    flows,_ = [half.split("\n") for half in inp.strip().split("\n\n")]
    workflows = defaultdict()
    for flow in flows:
        # px{a<2006:qkq,m>2090:A,rfg}
        label, rules = flow[:-1].split("{")
        tmp = []
        for rule in rules.split(","):
            if ":" in rule:
                regex = re.search(r"(?P<categ>.*)(?P<comp>[><])(?P<quant>[0-9]+):(?P<nxt>.*)", rule)
                tmp.append([regex.group("categ"), regex.group("comp"), int(regex.group("quant")), regex.group("nxt")])
            else:
                tmp.append([rule])
        workflows[label] = tmp
    
    return workflows

def split_ranges(workflows, minval, maxval):
    uncertainRanges = [("in", {ch: [minval, maxval] for ch in "xmas"})]
    acceptedRanges = []
    rejectedRanges = []
    while uncertainRanges:
        label, xmas = uncertainRanges.pop()
        thisFlow = workflows[label]

        for rating in thisFlow:
            if not xmas:
                break
            if len(rating) == 1:
                if rating[0] == "A":
                    acceptedRanges.append(xmas)
                elif len(rating) == 1 and rating[0] == "R":
                    rejectedRanges.append(xmas)
                else:
                    step = (rating[0], xmas)
                    uncertainRanges.append(step)
            else:
                categ, comp, quant, nxt = rating
                if comp == ">":
                    relevant_min, relevant_max = xmas[categ]
                    # s>2770:qs -> s: [3000,4000]
                    if relevant_min > quant:
                        if nxt == "A":      acceptedRanges.append(xmas)
                        elif nxt == "R":    rejectedRanges.append(xmas)
                        else:
                            step = (nxt, xmas)
                            uncertainRanges.append(step)
                    # s>2770:qs -> s : [2770, 4000] -> s: [2770, 2770], s: [2771, 4000]
                    # or s: [2770, 2770]
                    elif relevant_min == quant:
                        if relevant_max == quant:
                            step = (nxt, xmas)
                            uncertainRanges.append(step)
                        else:
                            xmas1 = copy.deepcopy(xmas)
                            xmas1[categ] = [relevant_min, relevant_min]
                            xmas2 = copy.deepcopy(xmas)
                            xmas2[categ] = [relevant_min+1, relevant_max]
                            step = (nxt, xmas2)
                        
                            if nxt == "A":      acceptedRanges.append(xmas2)
                            elif nxt == "R":    rejectedRanges.append(xmas2)
                            else:
                                step = (nxt, xmas2)
                                uncertainRanges.append(step)
                            xmas = xmas1
                    else:
                        # relevant_min < quant
                        # s>2770:qs -> s : [2500, 3000] -> s: [2500, 2770], s: [2771, 3000]
                        xmas1 = copy.deepcopy(xmas)
                        xmas1[categ] = [relevant_min, quant]
                        xmas2 = copy.deepcopy(xmas)
                        xmas2[categ] = [quant+1, relevant_max]
                        step = (nxt, xmas2)
                    
                        if nxt == "A":      acceptedRanges.append(xmas2)
                        elif nxt == "R":    rejectedRanges.append(xmas2)
                        else:
                            step = (nxt, xmas2)
                            uncertainRanges.append(step)
                        xmas = xmas1

                else: # "<"
                    relevant_min, relevant_max = xmas[categ]
                    # s<2770:qs -> s: [2000,2500]
                    if relevant_max < quant:
                        if nxt == "A":      acceptedRanges.append(xmas)
                        elif nxt == "R":    rejectedRanges.append(xmas)
                        else:
                            step = (nxt, xmas)
                            uncertainRanges.append(step)
                    # s<2770:qs -> s : [2000, 2770] -> s: [2000, 2769], s: [2770, 2770]
                    # or s: [2770, 2770]
                    elif relevant_max == quant:
                        if relevant_min == quant:
                            step = (nxt, xmas)
                            uncertainRanges.append(step)
                        else:
                            xmas1 = copy.deepcopy(xmas)
                            xmas1[categ] = [relevant_min, relevant_max-1]
                            xmas2 = copy.deepcopy(xmas)
                            xmas2[categ] = [relevant_max, relevant_max]
                            step = (nxt, xmas1)
                        
                            if nxt == "A":      acceptedRanges.append(xmas1)
                            elif nxt == "R":    rejectedRanges.append(xmas1)
                            else:
                                step = (nxt, xmas1)
                                uncertainRanges.append(step)
                            xmas = xmas2
                    else:
                        # relevant_max > quant
                        # s<2770:qs -> s : [2500, 3000] -> s: [2500, 2769], s: [2770, 3000]
                        xmas1 = copy.deepcopy(xmas)
                        xmas1[categ] = [relevant_min, quant-1]
                        xmas2 = copy.deepcopy(xmas)
                        xmas2[categ] = [quant, relevant_max]
                        step = (nxt, xmas1)
                    
                        if nxt == "A":      acceptedRanges.append(xmas1)
                        elif nxt == "R":    rejectedRanges.append(xmas1)
                        else:
                            step = (nxt, xmas1)
                            uncertainRanges.append(step)
                        xmas = xmas2
    return acceptedRanges
                    


def p2(inp):
    acceptedRanges = split_ranges(parseP2(inp), 1, 4000)
    res = 0
    for ar in acceptedRanges:
        X = ar["x"][1] - ar["x"][0] + 1 
        M = ar["m"][1] - ar["m"][0] + 1 
        A = ar["a"][1] - ar["a"][0] + 1 
        S = ar["s"][1] - ar["s"][0] + 1 
        res += X*M*A*S
    return res



In [50]:
# expected value: 167409079868000
p2(TEST)

167409079868000

In [51]:
p2(A)

125355665599537

In [ ]:
# REALLY need to come back and clean this up lol